Import the pandas library and open the HESA database:

In [4]:
import pandas
year_hesa = '2019-20'
year_other_data = 2019
data_path = f'path/fulltime_undergrads_{year_hesa}.csv'
hesa =  pandas.read_csv(data_path, index_col=0)

gdp_path = 'path/gdp_per_capita.csv'
gdp =  pandas.read_csv(gdp_path, index_col=0)

gini_path = 'path/gini_index.xls'
gini =  pandas.read_excel(gini_path, index_col=0, skiprows=3)

tert_exp_path = 'path/tertiary_education_expenditure.xls'
tert_exp =  pandas.read_excel(tert_exp_path, index_col=0, skiprows = 3)

tert_enr_path = 'path/tertiary_education_enrollment.xls'
tert_enr =  pandas.read_excel(tert_enr_path, index_col=0, skiprows=3)

pop_path = 'path/population.csv'
pop =  pandas.read_csv(pop_path, index_col=0, skiprows=4)

Select rows in to-be-merged-dataset

In [5]:
tert_enr.head()

,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,"School enrollment, tertiary (% gross)",SE.TER.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.725639,37.761131,16.195829,16.40542,15.27177,15.59921,NaN,NaN,NaN,NaN
Africa Eastern and Southern,AFE,"School enrollment, tertiary (% gross)",SE.TER.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Afghanistan,AFG,"School enrollment, tertiary (% gross)",SE.TER.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.622260,NaN,NaN,8.23068,NaN,NaN,NaN,9.68642,NaN,10.56252
Africa Western and Central,AFW,"School enrollment, tertiary (% gross)",SE.TER.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,AGO,"School enrollment, tertiary (% gross)",SE.TER.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.130300,NaN,8.838330,NaN,8.40094,9.33626,NaN,NaN,NaN,NaN


In [6]:
tert_enr_definitive = tert_enr[[f'{year_other_data}']]
tert_enr_definitive = tert_enr_definitive.rename(columns={f'{year_other_data}': 'Tertiary Education Enrollment'})

tert_exp_definitive = tert_exp[[f'{year_other_data}']]
tert_exp_definitive = tert_exp_definitive.rename(columns={f'{year_other_data}': 'Tertiary Expenditure'})

gini_definitive = gini[[f'{year_other_data}']]
gini_definitive = gini_definitive.rename(columns={f'{year_other_data}': 'GINI'})

pop_definitive = pop[[f'{year_other_data}']]
pop_definitive = gini_definitive.rename(columns={f'{year_other_data}': 'Population'})

Prepares GDP database for merging

In [7]:
gdp_step_1 = gdp.loc[gdp['Year'] == year_other_data]
gdp_definitive = gdp_step_1[['GDP per capita, PPP (constant 2017 international $)']]
gdp_definitive = gdp_definitive.rename(columns={'GDP per capita, PPP (constant 2017 international $)': 'GDP per Capita'})

Merge all dataframes one by one

In [8]:
hesa_gdp_joined = hesa.merge(gdp_definitive, left_index=True, right_index=True, how='left')
hesa_gdp_tertenr_joined = hesa_gdp_joined.merge(tert_enr_definitive, left_index=True, right_index=True, how='left')
hesa_gdp_tertenr_tertexp_joined = hesa_gdp_tertenr_joined.merge(tert_exp_definitive, left_index=True, right_index=True, how='left')
hesa_gdp_tertenr_tertexp_gini_joined = hesa_gdp_tertenr_tertexp_joined.merge(gini_definitive, left_index=True, right_index=True, how='left')

hesa_gdp_tertenr_tertexp_gini_joined = hesa_gdp_tertenr_tertexp_gini_joined.rename(columns={'Number': 'Number of Undergraduate Students'})
hesa_gdp_tertenr_tertexp_gini_pop_joined = hesa_gdp_tertenr_tertexp_gini_joined.merge(pop_definitive, left_index=True, right_index=True, how='left')

Export to csv

In [9]:
hesa_gdp_tertenr_tertexp_gini_pop_joined.to_csv(f'all_databases_merged_{year_other_data}.csv', index=True)